In [1]:
import os
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from collections import Counter
from nltk.corpus import stopwords
import nltk
import math
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\robin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\robin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
def preprocess(text):

    text = text.lower()
    tokens = word_tokenize(text)
    textnew = []
    stop_words = set(stopwords.words('english'))
    
    for i in tokens:
        if i not in stop_words:
            if i.isalnum():
                textnew.append(i)

    return textnew

In [3]:
directory = "Humor,Hist,Media,Food/"
files = os.listdir(directory)
document_tokens = []
pos_index = {}
documentid = {}
num = 1
for file_name in files: 
    documentid[num] = file_name
    
    file_text = open(directory+file_name,'r',encoding='ISO-8859-1').read()    # read the file
    tokens = preprocess(file_text)
    document_tokens.append(tokens)
    
    for pos,token in enumerate(tokens):
        if token not in pos_index:
            pos_index[token] = [0,{}]
            
        if num not in pos_index[token][1]:
            pos_index[token][0]+=1
            pos_index[token][1][num] = []
            
        pos_index[token][1][num].append(pos)
    num+=1
print(len(document_tokens))

1133


In [4]:
def jaccard_coefficient(doc_tokens,query,docid_list):
#     print(b,k,docid_list)
    jaccard_scores=[]
    num_of_docs = 5
    for i in range(len(doc_tokens)):
        count=0
        count_un = len(doc_tokens[i])+len(query)
        for j in doc_tokens[i]:
            if j in query:
                count+=1
                count_un-=1
        jaccard_scores.append([count/count_un,i])
    temp=sorted(jaccard_scores,reverse=True)
    print("TOP 5 DOCUMENTS BASAED ON JACCARD COEFFICINT")
    for i in range(num_of_docs):
        print(docid_list[temp[i][1]])

In [5]:

with open('Q1_input.txt','r') as inp:
    number_of_queries = int(inp.readline())
    for i in range(number_of_queries):
        query = inp.readline()
# query = "sixty two"
        query_token = preprocess(query)

        jaccard_coefficient(document_tokens,query_token,documentid )
        print()


TOP 5 DOCUMENTS BASAED ON JACCARD COEFFICINT
readme.bat
aclamt.txt
fascist.txt
deadlysins.txt
aussie.lng

TOP 5 DOCUMENTS BASAED ON JACCARD COEFFICINT
popmach
abbott.txt
hate.hum
robot.tes
commutin.jok

TOP 5 DOCUMENTS BASAED ON JACCARD COEFFICINT
acronym.lis
ambrose.bie
psalm_nixon
psalm23.txt
prover_w.iso

TOP 5 DOCUMENTS BASAED ON JACCARD COEFFICINT
acronym.lis
flattax.hum
insult.lst
collected_quotes.txt
oxymoron.jok

TOP 5 DOCUMENTS BASAED ON JACCARD COEFFICINT
electric.txt
addrmeri.txt
lampoon.jok
policpig.hum
polemom.txt

TOP 5 DOCUMENTS BASAED ON JACCARD COEFFICINT
welfare
t-shirt.hum
aeonint.txt
libraway.txt
weights.hum

TOP 5 DOCUMENTS BASAED ON JACCARD COEFFICINT
abbott.txt
acronyms.txt
lifeonledge.txt
tribble.hum
gd_frasr.txt

TOP 5 DOCUMENTS BASAED ON JACCARD COEFFICINT
oilgluts.hum
normal.boy
nintendo.jok
lawskool.txt
cucumber.jok

TOP 5 DOCUMENTS BASAED ON JACCARD COEFFICINT
recipe.001
t-10.hum
arnold.txt
chung.iv
making_y.wel

TOP 5 DOCUMENTS BASAED ON JACCARD COEFFICINT

In [6]:
def calculate_idf(count,N):
    return (math.log(N/(1+count)))
def raw_tf(count):
    return count
def log_norm_tf(count):
    return math.log(count+1)
def binary_tf(count):
    return (1 if count>0 else 0)
def double_norm_tf(count,maxcount):
    return (0.5+0.5*(count/float(maxcount)))
def term_freq_tf(count,sum_total):
    return (count/sum_total)

In [7]:
def tf_idf_retrieval(body_list,query,k,documentname_list,pos_index,variant):
    idf={}
    for i in pos_index:

        idf[i]=calculate_idf(pos_index[i][0],len(documentname_list))

    tf_dict=calculate_tf_score(body_list,variant)
#     print(tf_dict[0])
    
    tf_idf_matrix = []
    for i in range(len(body_list)):
        tf_idf_matrix.append({})
#     print(tf_idf_matrix)
    for i in range(len(body_list)):
        for j in idf:
#             print(idf[j])
            if j in tf_dict[i]:
                tf_idf_matrix[i][j] = tf_dict[i][j]*idf[j]
            else:
                tf_idf_matrix[i][j] = 0
    return tf_idf_matrix


In [14]:
def calculate_tf_score(body_list,variant):
    tf_dict={}
    for i in range(len(body_list)):
        term_counts = dict(Counter(body_list[i]))

        max_tf = 0
        sum_tf=0
        for j in term_counts:
            sum_tf+=term_counts[j]
            if term_counts[j]>max_tf:
                max_tf = term_counts[j]
        for j in term_counts:
            if variant==0:
                term_counts[j] = raw_tf(term_counts[j])
            elif variant==1:
                term_counts[j] = log_norm_tf(term_counts[j])
            elif variant==2:
                term_counts[j] = binary_tf(term_counts[j])
            elif variant==3:
                term_counts[j] = double_norm_tf(term_counts[j],max_tf)
            elif variant==4:
                term_counts[j] = term_freq_tf(term_counts[j],sum_tf)

        tf_dict[i] = term_counts
#     print(tf_dict)

    return tf_dict

In [16]:
var = ['raw_tf','long_norm_tf','binary_tf','double_norm_tf','term_freq_tf']
with open('Q1_input.txt','r') as inp:
    number_of_queries = int(inp.readline())
    for i in range(number_of_queries):
        query = inp.readline()
        print(query)
        query_token = preprocess(query)
        i=0
        for k in range(5):
            scores = []
            matrix = tf_idf_retrieval(document_tokens,query_token,5,documentid,pos_index,k)
            for doc in matrix:
            #     print(len(doc))
                s = 0
                for q in query_token:
                    if q in doc:
                        s = s + doc[q]
                scores.append([s,i])
                i = i+1
            scores = sorted(scores, reverse = True)    
            print("TOP 5 DOCUMENTS BASED ON tf_idf COEFFICINT for variant: " + var[k])
            for i in range(5):
                if(scores[i][0]>0):
                    print("score: " ,scores[i][0], "dcoument name: ",documentid[scores[i][1]+1])
                    
                    
            print()
    
        
    

A series of death at a hospital for elderly patients in shanghai is underscoring the dangerous consequences of china's stuborn pursuit of a zero-COVID approach

TOP 5 DOCUMENTS BASED ON tf_idf COEFFICINT for variant: raw_tf
score:  148.09094171375654 dcoument name:  acne1.txt
score:  92.41812005625235 dcoument name:  antibiot.txt
score:  85.20447411346699 dcoument name:  stuf11.txt
score:  83.35619184998033 dcoument name:  humor9.txt
score:  74.75182466495792 dcoument name:  stuf10.txt

TOP 5 DOCUMENTS BASED ON tf_idf COEFFICINT for variant: long_norm_tf
score:  29.09361496457005 dcoument name:  idaho.txt
score:  26.48074051971883 dcoument name:  mog-history
score:  20.31994089023259 dcoument name:  marriage.hum
score:  17.194980411584098 dcoument name:  prooftec.txt
score:  15.60353053984451 dcoument name:  variety2.asc

TOP 5 DOCUMENTS BASED ON tf_idf COEFFICINT for variant: binary_tf
score:  22.671642488757556 dcoument name:  mog-history
score:  20.157018921461656 dcoument name:  id